<a href="https://colab.research.google.com/github/webb-e/S2_Landsat_Comparison/blob/main/ConcatRegions_cleandata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## import libraries
import numpy as np
import pandas as pd
import glob


In [ ]:
# List all files starting with 'analysis ready'
files = glob.glob('ALPOD/Lakewise_csvs/analysis_ready_*')

# Read and concatenate all the files
df = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

# Display the combined DataFrame
print(df.head())

In [ ]:
### remove lakes where S2 max is zero, or where Pickens and Pekel are both zero
### these tend to not be real lakes
data = df[df['S2max'] != 0]
data = data[~((data['Landsat_Pekel'] == 0) & (data['Landsat_Pickens'] == 0))]

In [ ]:
## get the sum of water for each year in each region
sum_by_year = data.groupby(['year', 'region'])['S2max'].sum().reset_index()
### get the percentage of the year with the maximum water extent
sum_by_year['max_value'] = sum_by_year.groupby(['region'])['S2max'].transform('max')
sum_by_year['proportion_of_max'] = (sum_by_year['S2max'] / sum_by_year['max_value'])
## join back with original dataframe
dataframe = data.merge(sum_by_year[['year', 'region', 'proportion_of_max']], on=['year', 'region'], how='left')
dataframe.head()

In [ ]:
## remove lakes where error is >4 standard deviations from the mean
print(f"Number of rows before: {len(dataframe)}")
errorcols = ['Pickens_error_abs', 'Pekel_error_abs']

# Apply the transformation
for col in errorcols:
    col_mean = dataframe[col].mean(skipna=True)
    col_std = dataframe[col].std(skipna=True)
    # Keep rows where values are within 4 standard deviations
    dat = dataframe[(dataframe[col] - col_mean).abs() <= 4 * col_std]

print(f"Number of rows after: {len(dat)}")

In [ ]:
print(f"Number of rows before: {len(dat)}")
filtered_data = (
    dat
    # Filter rows where specified columns are not NaN
    .dropna(subset=["Landsat_Pekel", "Landsat_Pickens"])
    # Group by `lake_id`
    .groupby("lake_id")
    # Filter groups with at least 6 rows
    .filter(lambda group: len(group) >= 6)
    # Ungroup by resetting the index
    .reset_index(drop=True)
)
print(f"Number of rows after: {len(filtered_data)}")

In [ ]:
## save to csv
filtered_data.to_csv('ALPOD/Lakewise_csvs/Landsat_analysis_data.csv', index=False)